In [1]:
import numpy as np
from gurobipy import Model, GRB, quicksum, LinExpr
import pandas as pd
import numpy as np
from utils.is_efficient import is_efficient
from models.modelsDEA import DEA
from utils.groupDMUs import groupDMUs


In [2]:
y = np.array([
    [70, 103, 100, 80],
    [60, 125, 90, 90],
    [50, 110, 105, 85],
    [80, 80, 110, 90],
    [56, 40, 60, 55],
    [40, 95, 120, 110],
    [100, 180, 200, 210],
    [25, 55, 180, 160],
    [65, 150, 125, 145],
    [40, 110, 70, 115],
    [70, 117, 122, 115],
    [92, 135, 89, 64],
    [88, 47, 57, 109],
    [48, 68, 146, 99],
    [79, 123, 220, 122],
    [99, 114, 89, 49],
    [97, 101, 88, 55],
    [55, 55, 132, 116],
    [80, 97, 142, 168],
    [97, 68, 209, 122],
    [70, 112, 117, 0],
    [99, 134, 134, 0],
    [68, 89, 178, 0],
    [107, 89, 136, 0],
    [78, 125, 58, 0],
    [88, 55, 112, 0],
    [67, 45, 83, 0],
    [45, 78, 61, 0],
    [57, 134, 77, 0],
    [45, 80, 142, 0],
    [35, 95, 136, 0],
    [89, 125, 267, 0]
])

x = np.array([
    [30, 15, 10, 15],
    [40, 12, 9, 18],
    [35, 16, 9.7, 10],
    [38, 20, 8.5, 8.5],
    [28, 9, 11, 12.5],
    [37, 13, 7.6, 14],
    [31, 18, 8.3, 11],
    [35, 15, 10, 15],
    [25, 19, 9.5, 19],
    [30, 10, 6.5, 21],
    [25, 12, 8, 12],
    [45, 15, 6.6, 23],
    [35, 13, 12, 20.5],
    [32, 16, 6.8, 11.2],
    [26, 22, 8.7, 15.6],
    [19, 16, 8.5, 12.4],
    [25, 24, 6, 8.8],
    [32, 18, 5.5, 6.8],
    [33, 8, 7.8, 13.4],
    [27, 10, 8.7, 21.6],
    [16, 6, 3.5, 11],
    [18, 7, 4.2, 14.5],
    [26, 11, 6.7, 20.9],
    [34, 12, 5.7, 7.8],
    [21, 15, 3.8, 6.9],
    [22, 13, 5.9, 14.2],
    [18, 9, 7.8, 11.4],
    [14, 10, 11.2, 5.6],
    [27, 10, 7.9, 13.5],
    [26, 15, 8.5, 10.9],
    [31, 17, 8.2, 11.2],
    [32, 14, 4.5, 12.3]
])

In [3]:
N, R, dmu_outputs, correspondingR, L, dmu_to_subgroup, M = groupDMUs(y)


In [4]:
(n, m), s = x.shape, y.shape[1]

In [5]:
n, m, s

(32, 4, 4)

In [6]:
alphas = []
com = []
com1 = []
com2 = []
com3 = []

a, b = [0.6, 0.15], [0.85, 0.40]
V = []
for knot in list(N[0]):
    common = ""
    common1 = ''
    print("knot: ", knot)
    model = Model("non_homo_crs")
    gamma = {}

    for i in range(m):
        for k in range(len(R)):  
            gamma[(i, k, 0)] = model.addVar(vtype=GRB.CONTINUOUS, lb=0)

    for i in range(m):
        L_np = L[0]
        for k in range(len(R)):
            if k not in L_np:
                model.addConstr(gamma[(i, k, 0)] == 0)

    for i in range(m):
        L_np = L[0]
        for k in range(len(R)):
            if k in L_np:
                model.addConstr(gamma[(i, k, 0)] >= 0.0001)

    model.setParam(GRB.Param.OutputFlag, 0)

    mu = [model.addVar(lb=0.0001, name=f"mu_{r}") for r in range(s)]
    v = [model.addVar(lb=0.00025, name=f"v_{i}") for i in range(m)]

    L_npo = L[0]

    eo = LinExpr()
    estr = ""
    for k in L_npo:
        for r in list(R[k]): ###########
            eo += mu[r] * y[knot, r]
            estr += f"μ{r}y{knot, r}  "
    model.setObjective(eo, GRB.MAXIMIZE)
    print("objective: ", estr)

    nstr = ""
    norm_expr = LinExpr()
    for i in range(m):
        norm_expr += v[i] * x[knot, i]
        nstr += f"v{i}x{knot, i}  "
    model.addConstr(norm_expr == 1, name="Normalization")
    print('normalization: ', nstr)

    print("\n")

    numc = 0
    for j in list(N[0]): # \forall j in N1
        L_np = list(L[0])
        common += f"\n\nDMU: {j}\n"
        for k in L_np:
            common += f"k: {k}\n"
            output_expr = LinExpr()
            input_expr = LinExpr()
            out, inp = "", ""
            for r in list(R[k]):
                output_expr += mu[r] * y[j, r]
                out+= f"μ{r}y{j, r}  "

            for i in range(m):
                input_expr += gamma[(i, k, 0)] * x[j, i]
                inp += f"γ{i, k, 0}x{j,i}  "

            model.addConstr(output_expr - input_expr <= 0, name=f"Constraint_Rk_{k}")
            common += f"{inp}  -  {out}\n"
            numc += 1
    com.append(common)

    numv = 0
    for j in list(N[1]): # \forall j in N1
        common1 += f"\n\nDMU: {j}\n"
        L_np = list(L[1])
        for k in L_np:
            common1 += f"k: {k}\n"
            output_expr = LinExpr()
            input_expr = LinExpr()
            out, inp = "", ""
            for r in list(R[k]):
                output_expr += mu[r] * y[j, r]
                out+= f"μ{r}y{j, r}  "

            for i in range(m):
                input_expr += v[i] * x[j, i]
                inp += f"γ{i, k, 0}x{j,i}  "

            model.addConstr(output_expr - input_expr <= 0, name=f"Constraint_Rk_{k}")
            numv += 1
            common1 += f"{inp}  -  {out}\n"
            
    com1.append(common1)

    bi = ""
    for i in range(m):
        L_np = list(L[0])
        bound_inp = LinExpr()
        bi += f"\n i == {i}\n"
        for k in L_np:
            bound_inp += gamma[(i, k, 0)]
            bi += f"γ{i, k, 0}  "
        model.addConstr(bound_inp == v[i])
        bi += f" = v{i}"
    com2.append(bi)


    bds = ""
    for i in range(m):
        np_j = 0
        for k in list(L[np_j]):
            model.addConstr(a[k]* v[i] <= gamma[(i, k, np_j)])
            model.addConstr(gamma[(i, k, np_j)] <= v[i] * b[k])
            bds += f"gamma{i, k, np_j} ∈ {a[k], b[k]} \n"
    com3.append(bds)


    model.optimize()

    if model.status == GRB.OPTIMAL:
        print("Optimal solution found.")
    elif model.status == GRB.INFEASIBLE:
        print("Model is infeasible.")
    elif model.status == GRB.UNBOUNDED:
        print("Model is unbounded.")
    elif model.status == GRB.TIME_LIMIT:
        print("Time limit reached.")
    elif model.status == GRB.INTERRUPTED:
        print("Optimization interrupted.")
    elif model.status == GRB.SUBOPTIMAL:
        print("Suboptimal solution found.")
    else:
        print("Optimization ended with an unknown status.")
    
    v_dash = [vee.x for vee in v]
    V.append(v_dash)
    
    alphas.append({
        "DMU": knot,
        "alphas": {key: (gamma[key].x) / (v[key[0]].x) for key in gamma if key[2] == dmu_to_subgroup[knot]}
    })

knot:  0
Set parameter WLSAccessID
Set parameter WLSSecret
Set parameter LicenseID to value 2579375
Academic license 2579375 - for non-commercial use only - registered to ma___@iitd.ac.in
objective:  μ0y(0, 0)  μ1y(0, 1)  μ2y(0, 2)  μ3y(0, 3)  
normalization:  v0x(0, 0)  v1x(0, 1)  v2x(0, 2)  v3x(0, 3)  


Optimal solution found.
knot:  1
objective:  μ0y(1, 0)  μ1y(1, 1)  μ2y(1, 2)  μ3y(1, 3)  
normalization:  v0x(1, 0)  v1x(1, 1)  v2x(1, 2)  v3x(1, 3)  


Optimal solution found.
knot:  2
objective:  μ0y(2, 0)  μ1y(2, 1)  μ2y(2, 2)  μ3y(2, 3)  
normalization:  v0x(2, 0)  v1x(2, 1)  v2x(2, 2)  v3x(2, 3)  


Optimal solution found.
knot:  3
objective:  μ0y(3, 0)  μ1y(3, 1)  μ2y(3, 2)  μ3y(3, 3)  
normalization:  v0x(3, 0)  v1x(3, 1)  v2x(3, 2)  v3x(3, 3)  


Optimal solution found.
knot:  4
objective:  μ0y(4, 0)  μ1y(4, 1)  μ2y(4, 2)  μ3y(4, 3)  
normalization:  v0x(4, 0)  v1x(4, 1)  v2x(4, 2)  v3x(4, 3)  


Optimal solution found.
knot:  5
objective:  μ0y(5, 0)  μ1y(5, 1)  μ2y(5, 2)  μ3

In [7]:
V[0]

[0.004440701671048473, 0.02949517233115363, 0.00025, 0.02812342432674943]

In [8]:
print(com[6])



DMU: 0
k: 0
γ(0, 0, 0)x(0, 0)  γ(1, 0, 0)x(0, 1)  γ(2, 0, 0)x(0, 2)  γ(3, 0, 0)x(0, 3)    -  μ0y(0, 0)  μ1y(0, 1)  μ2y(0, 2)  
k: 1
γ(0, 1, 0)x(0, 0)  γ(1, 1, 0)x(0, 1)  γ(2, 1, 0)x(0, 2)  γ(3, 1, 0)x(0, 3)    -  μ3y(0, 3)  


DMU: 1
k: 0
γ(0, 0, 0)x(1, 0)  γ(1, 0, 0)x(1, 1)  γ(2, 0, 0)x(1, 2)  γ(3, 0, 0)x(1, 3)    -  μ0y(1, 0)  μ1y(1, 1)  μ2y(1, 2)  
k: 1
γ(0, 1, 0)x(1, 0)  γ(1, 1, 0)x(1, 1)  γ(2, 1, 0)x(1, 2)  γ(3, 1, 0)x(1, 3)    -  μ3y(1, 3)  


DMU: 2
k: 0
γ(0, 0, 0)x(2, 0)  γ(1, 0, 0)x(2, 1)  γ(2, 0, 0)x(2, 2)  γ(3, 0, 0)x(2, 3)    -  μ0y(2, 0)  μ1y(2, 1)  μ2y(2, 2)  
k: 1
γ(0, 1, 0)x(2, 0)  γ(1, 1, 0)x(2, 1)  γ(2, 1, 0)x(2, 2)  γ(3, 1, 0)x(2, 3)    -  μ3y(2, 3)  


DMU: 3
k: 0
γ(0, 0, 0)x(3, 0)  γ(1, 0, 0)x(3, 1)  γ(2, 0, 0)x(3, 2)  γ(3, 0, 0)x(3, 3)    -  μ0y(3, 0)  μ1y(3, 1)  μ2y(3, 2)  
k: 1
γ(0, 1, 0)x(3, 0)  γ(1, 1, 0)x(3, 1)  γ(2, 1, 0)x(3, 2)  γ(3, 1, 0)x(3, 3)    -  μ3y(3, 3)  


DMU: 4
k: 0
γ(0, 0, 0)x(4, 0)  γ(1, 0, 0)x(4, 1)  γ(2, 0, 0)x(4, 2)  γ(3, 0, 0)x(4, 3)    

In [9]:
print(com2[6])


 i == 0
γ(0, 0, 0)  γ(0, 1, 0)   = v0
 i == 1
γ(1, 0, 0)  γ(1, 1, 0)   = v1
 i == 2
γ(2, 0, 0)  γ(2, 1, 0)   = v2
 i == 3
γ(3, 0, 0)  γ(3, 1, 0)   = v3


In [10]:
print(com3[6])

gamma(0, 0, 0) ∈ (0.6, 0.85) 
gamma(0, 1, 0) ∈ (0.15, 0.4) 
gamma(1, 0, 0) ∈ (0.6, 0.85) 
gamma(1, 1, 0) ∈ (0.15, 0.4) 
gamma(2, 0, 0) ∈ (0.6, 0.85) 
gamma(2, 1, 0) ∈ (0.15, 0.4) 
gamma(3, 0, 0) ∈ (0.6, 0.85) 
gamma(3, 1, 0) ∈ (0.15, 0.4) 



In [11]:
for xyz in range(20):
    print((com[0] == com[xyz]) and (com1[0] == com1[xyz]) and (com2[0] == com2[xyz]) and (com3[0] == com3[xyz]))

True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True


In [12]:
alphas

[{'DMU': 0,
  'alphas': {(0, 0, 0): 0.6,
   (0, 1, 0): 0.4,
   (1, 0, 0): 0.7652043541099659,
   (1, 1, 0): 0.23479564589003404,
   (2, 0, 0): 0.6000000000000001,
   (2, 1, 0): 0.4,
   (3, 0, 0): 0.7708805979312864,
   (3, 1, 0): 0.2291194020687136}},
 {'DMU': 1,
  'alphas': {(0, 0, 0): 0.6000000000000001,
   (0, 1, 0): 0.4,
   (1, 0, 0): 0.7107198392302029,
   (1, 1, 0): 0.28928016076979707,
   (2, 0, 0): 0.6000000000000001,
   (2, 1, 0): 0.4,
   (3, 0, 0): 0.6,
   (3, 1, 0): 0.4}},
 {'DMU': 2,
  'alphas': {(0, 0, 0): 0.6000000000000001,
   (0, 1, 0): 0.4,
   (1, 0, 0): 0.85,
   (1, 1, 0): 0.15000000000000002,
   (2, 0, 0): 0.6000000000000001,
   (2, 1, 0): 0.4,
   (3, 0, 0): 0.85,
   (3, 1, 0): 0.14999999999999997}},
 {'DMU': 3,
  'alphas': {(0, 0, 0): 0.6000000000000001,
   (0, 1, 0): 0.4,
   (1, 0, 0): 0.6000000000000001,
   (1, 1, 0): 0.4,
   (2, 0, 0): 0.6,
   (2, 1, 0): 0.4000000000000001,
   (3, 0, 0): 0.7577011175961557,
   (3, 1, 0): 0.24229888240384428}},
 {'DMU': 4,
  'alph

In [13]:
for knot2 in list(N[1]):
    ans = {}
    for i in range(m):
        ans[(i, 0, 1)] = 1.0
        ans[(i, 1, 1)] = 0.0
    alphas.append({
        "DMU": knot,
        "alphas": ans
    })

In [14]:
numc, numv

(40, 12)

In [15]:
20 * 2, 12*1

(40, 12)

In [16]:
len(alphas)

32

In [17]:
x.shape

(32, 4)

In [18]:
new_x = np.zeros((n, m, len(R)))
for j in range(n):
    alpha_values = alphas[j]['alphas'].items()
    for (i, k, p) , alpha in alpha_values:
        new_x[j, i, k] = alpha * x[j, i]


In [19]:
new_x[21]

array([[18. ,  0. ],
       [ 7. ,  0. ],
       [ 4.2,  0. ],
       [14.5,  0. ]])

In [20]:
alphas[21]

{'DMU': 19,
 'alphas': {(0, 0, 1): 1.0,
  (0, 1, 1): 0.0,
  (1, 0, 1): 1.0,
  (1, 1, 1): 0.0,
  (2, 0, 1): 1.0,
  (2, 1, 1): 0.0,
  (3, 0, 1): 1.0,
  (3, 1, 1): 0.0}}

In [21]:
x[21]

array([18. ,  7. ,  4.2, 14.5])

In [22]:
new_x.shape

(32, 4, 2)

In [23]:
ccr_res= []

In [24]:
k = 0
x_dash = new_x[list(M[k]) , : , k]
y_dash = y[list(M[k]), :] 
y_dash = y_dash[:, list(R[k])]

dea = DEA(x_dash, y_dash)
ccr_res.append(dea.ccr_input()[["DMU", 'efficiency']])

In [25]:
k = 1
x_dash = new_x[list(M[k]) , : , k]
y_dash = y[list(M[k]), :] 
y_dash = y_dash[:, list(R[k])]

dea = DEA(x_dash, y_dash)
ccr_res.append(dea.ccr_input()[["DMU", 'efficiency']])


In [26]:
x_dash.shape

(20, 4)

In [27]:
y_dash

array([[ 80],
       [ 90],
       [ 85],
       [ 90],
       [ 55],
       [110],
       [210],
       [160],
       [145],
       [115],
       [115],
       [ 64],
       [109],
       [ 99],
       [122],
       [ 49],
       [ 55],
       [116],
       [168],
       [122]])

In [28]:
ccr_res[0].iloc[11]['efficiency']

np.float64(1.0)

In [29]:
alphas[7]

{'DMU': 7,
 'alphas': {(0, 0, 0): 0.6,
  (0, 1, 0): 0.39999999999999997,
  (1, 0, 0): 0.731940025992614,
  (1, 1, 0): 0.268059974007386,
  (2, 0, 0): 0.6000000000000001,
  (2, 1, 0): 0.4,
  (3, 0, 0): 0.6220714223492086,
  (3, 1, 0): 0.37792857765079135}}

In [30]:
new_x[1]

array([[24.        , 16.        ],
       [ 8.52863807,  3.47136193],
       [ 5.4       ,  3.6       ],
       [10.8       ,  7.2       ]])

In [31]:
for k in range(20):
    ccr_res[0].at[k, 'efficiency'] += ccr_res[1].at[k, 'efficiency']
    ccr_res[0].at[k, 'efficiency'] /= 2


In [32]:
ccr_res[0]

,DMU,efficiency
0,0,0.546575
1,1,0.569979
2,2,0.637377
3,3,0.602034
4,4,0.529248
5,5,0.604526
6,6,0.824283
7,7,0.723197
8,8,0.873250
9,9,0.688625


In [33]:
ccr_res[1]

,DMU,efficiency
0,0,0.406794
1,1,0.326183
2,2,0.733484
3,3,0.384250
4,4,0.405499
5,5,0.606907
6,6,0.648566
7,7,0.610464
8,8,0.951850
9,9,0.422937


In [34]:
# Non Homo
#? EAT
#! FDH